In [173]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import re

df = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/Mining-BTC-180.csv"
)

headerColor = "#1a1a1a"
headerFontColor = "white"
rowEvenColor = "#f5f5f5"
rowOddColor = "white"
borderColor = "#e0e0e0"

for i, row in enumerate(df["Date"]):
    p = re.compile(" 00:00:00")
    datetime = p.split(df["Date"][i])[0]
    df.iloc[i, 1] = datetime

fig = make_subplots(
    shared_xaxes=True,
    rows=3,
    cols=2,
    specs=[
        [{"type": "table"}, {"type": "table"}],
        [{"type": "scatter", "colspan": 2}, None],
        [{"type": "scatter", "colspan": 2}, None],
    ],
)

fig.add_trace(
    go.Scatter(
        x=df["Date"], y=df["Mining-revenue-USD"], mode="lines", name="mining revenue"
    ),
    row=3,
    col=1,
)

fig.add_trace(
    go.Scatter(x=df["Date"], y=df["Hash-rate"], mode="lines", name="hash-rate-TH/s"),
    row=2,
    col=1,
)

fig.add_trace(
    go.Table(
        header=dict(
            values=[
                "Date",
                "Number<br>Transactions",
                "Output<br>Volume (BTC)",
                "Market<br>Price",
                "Hash<br>Rate",
                "Cost per<br>trans-USD",
                "Mining<br>Revenue-USD",
                "Trasaction<br>fees-BTC",
            ],
            font=dict(size=10),
            align="left",
            line_color=borderColor,
            line_width=1,
        ),
        cells=dict(
            values=[df[k].tolist() for k in df.columns[1:]],
            fill_color=[[rowOddColor, rowEvenColor] * len(df)],
            height=30,
            line_color=borderColor,
            line_width=1,
            align="left",
        ),
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Table(
        header=dict(
            values=[
                "Date",
                "Number<br>Transactions",
                "Output<br>Volume (BTC)",
                "Market<br>Price",
                "Hash<br>Rate",
                "Cost per<br>trans-USD",
                "Mining<br>Revenue-USD",
                "Trasaction<br>fees-BTC",
            ],
            font=dict(size=10),
            align="left",
            line_color=borderColor,
            line_width=1,
        ),
        cells=dict(
            values=[df[k].tolist() for k in df.columns[1:]],
            fill_color=[[rowOddColor, rowEvenColor] * len(df)],
            height=30,
            align="left",
            line_color=borderColor,
            line_width=1,
        ),
    ),
    row=1,
    col=2,
)

fig.update_layout(
    height=800,
    autosize=True,
    width=None,  # Remove any fixed width
    showlegend=False,
)

fig.show()

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [174]:
import os
import webbrowser
from jinja2 import Environment, FileSystemLoader
from dataclasses import dataclass


@dataclass
class Jinjja:
    def render(self):
        template_dir = os.path.join(os.path.abspath(os.getcwd()), "templates")
        env = Environment(loader=FileSystemLoader(template_dir))
        template = env.get_template("template.html.j2")

        overview_html = fig.to_html(
            full_html=False,  # Changed
            include_plotlyjs=True,
            config={"responsive": True},
            div_id="overview-plot",
        )

        fees_html = fig.to_html(
            full_html=False,  # Changed from True
            include_plotlyjs=False,  # Changed - only load once
            config={"responsive": True},
            div_id="fees-plot",
        )

        pnl_html = fig.to_html(
            full_html=False,  # Changed from True
            include_plotlyjs=False,  # Changed - only load once
            config={"responsive": True},
            div_id="pnl-plot",
        )

        overnight_html = fig.to_html(
            full_html=False,  # Same
            include_plotlyjs=False,  # Changed - only load once
            config={"responsive": True},
            div_id="overnight-plot",
        )

        # Render template with variables
        html_rendered = template.render(
            fees_html=fees_html,
            overview_html=overview_html,
            pnl_html=pnl_html,
            overnight_html=overnight_html,
        )
        html_report = html_rendered
        path = "/tmp/backtest_report.html"
        browser = True
        with open(path, "w") as html_file:
            html_file.write(html_report)

        if browser:
            webbrowser.open(f"file://{path}")

        # def in_jupyter_notebook():
        #     try:
        #         from IPython import get_ipython
        #         shell = get_ipython().__class__.__name__
        #         return shell == 'ZMQInteractiveShell'
        #     except (NameError, ImportError):
        #         return False

        # if in_jupyter_notebook():
        #     display(HTML(html_report))
        else:
            webbrowser.open(f"file://{path}")


j = Jinjja()
j.render()

In [ ]:
# from dash import Dash, dcc, html

# app = Dash()
# app.layout = html.Div([
#     dcc.Graph(figure=fig)
# ])

# app.run(debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter